In [1]:
import pandas as pd

In [2]:
import torch
import numpy as np

In [3]:
train = pd.read_json("dataset/training/EXIST2023_training.json", orient="index")

In [4]:
train = train.sample(frac=1).reset_index(drop=True)

In [5]:
train

,id_EXIST,lang,tweet,number_annotators,annotators,gender_annotators,age_annotators,labels_task1,labels_task2,labels_task3,split
0,201820,en,I need a man that’s obsessed with me because I...,6,"[Annotator_549, Annotator_550, Annotator_551, ...","[F, F, M, M, M, F]","[18-22, 23-45, 18-22, 23-45, 46+, 46+]","[NO, YES, NO, NO, NO, NO]","[-, DIRECT, -, -, -, -]","[[-], [OBJECTIFICATION], [-], [-], [-], [-]]",TRAIN_EN
1,101904,es,Otorga Ayuntamiento de Puebla donación de 600 ...,6,"[Annotator_175, Annotator_176, Annotator_177, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 46+, 23-45, 18-22]","[NO, NO, NO, NO, NO, NO]","[-, -, -, -, -, -]","[[-], [-], [-], [-], [-], [-]]",TRAIN_ES
2,202546,en,"@DrProudman Not sure what sexual violence is, ...",6,"[Annotator_617, Annotator_618, Annotator_619, ...","[F, F, M, M, M, F]","[18-22, 23-45, 18-22, 23-45, 46+, 46+]","[NO, YES, YES, YES, YES, YES]","[-, DIRECT, REPORTED, DIRECT, REPORTED, REPORTED]","[[-], [STEREOTYPING-DOMINANCE], [SEXUAL-VIOLEN...",TRAIN_EN
3,200846,en,"Once made equal to man, woman becomes his supe...",6,"[Annotator_397, Annotator_398, Annotator_399, ...","[F, F, M, M, M, F]","[18-22, 23-45, 18-22, 23-45, 46+, 46+]","[YES, YES, NO, YES, YES, NO]","[DIRECT, DIRECT, -, DIRECT, DIRECT, -]","[[IDEOLOGICAL-INEQUALITY], [IDEOLOGICAL-INEQUA...",TRAIN_EN
4,202057,en,Someone sounds embarrassed that their phalloce...,6,"[Annotator_674, Annotator_675, Annotator_676, ...","[F, F, M, M, M, F]","[18-22, 23-45, 18-22, 23-45, 46+, 46+]","[NO, NO, NO, YES, NO, NO]","[-, -, -, JUDGEMENTAL, -, -]","[[-], [-], [-], [IDEOLOGICAL-INEQUALITY, STERE...",TRAIN_EN
...,...,...,...,...,...,...,...,...,...,...,...
6915,101578,es,@levhita @nochaveznada Me parece que tú aproxi...,6,"[Annotator_73, Annotator_74, Annotator_75, Ann...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 46+, 23-45, 18-22]","[NO, NO, NO, YES, NO, YES]","[-, -, -, REPORTED, -, JUDGEMENTAL]","[[-], [-], [-], [OBJECTIFICATION], [-], [IDEOL...",TRAIN_ES
6916,102050,es,@VioIettiTahti @bio_beatriz @alvaro_delgado ¿P...,6,"[Annotator_241, Annotator_242, Annotator_243, ...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 46+, 23-45, 18-22]","[YES, YES, YES, YES, YES, YES]","[DIRECT, DIRECT, JUDGEMENTAL, DIRECT, DIRECT, ...","[[IDEOLOGICAL-INEQUALITY], [IDEOLOGICAL-INEQUA...",TRAIN_ES
6917,101252,es,Cuanto antes se introduzca la educación afecti...,6,"[Annotator_37, Annotator_38, Annotator_39, Ann...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 46+, 23-45, 18-22]","[NO, NO, NO, NO, YES, YES]","[-, -, -, -, JUDGEMENTAL, REPORTED]","[[-], [-], [-], [-], [UNKNOWN], [STEREOTYPING-...",TRAIN_ES
6918,103440,es,@DACoupleSW1 Y conocen muc@s trans con quién i...,6,"[Annotator_7, Annotator_8, Annotator_9, Annota...","[F, F, F, M, M, M]","[18-22, 23-45, 46+, 46+, 23-45, 18-22]","[NO, NO, NO, NO, NO, NO]","[-, -, -, -, -, -]","[[-], [-], [-], [-], [-], [-]]",TRAIN_ES


In [6]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")
model = AutoModelForSequenceClassification.from_pretrained(
    "xlm-roberta-base", num_labels=2
)

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense

In [7]:
training_columns = train.columns

In [8]:
training_columns

Index(['id_EXIST', 'lang', 'tweet', 'number_annotators', 'annotators',
       'gender_annotators', 'age_annotators', 'labels_task1', 'labels_task2',
       'labels_task3', 'split'],
      dtype='object')

In [9]:
text = train["tweet"]

In [10]:
encoded_text = [tokenizer(sent, return_tensors="pt") for sent in text]

In [11]:
encoded_text

[{'input_ids': tensor([[     0,     87,   3871,     10,    332,    450,     26,      7, 107250,
            5281,    678,    163,   6637,     87,     26,     39, 132294,    186,
          107250,   5281,    678,   4049,    221,      5,     27,     87,    831,
              26,     18,     54,     10,  10176,  23552,  33946,  13580,      6,
          232773,      2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
          1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])},
 {'input_ids': tensor([[     0,  37098,  32654, 144355,      8,   3561,  62175,   2301,   4117,
               8,   6331,   2717, 102523,     10,     21,  36676, 101162,      5,
            5655,  89045,     10,  26843,      4,     21, 125769,    121,     21,
          141096,  12409,   1832,   4384,   4501,      8,  71774,     86,    516,
            1148,   9991,   1496,  24448,      7, 136652,      7,     10,   3357,
           48136,     31,    113,      8,  27506, 

In [12]:
output = model(**encoded_text[0])

In [13]:
output.logits

tensor([[ 0.1312, -0.3700]], grad_fn=<AddmmBackward0>)

# Code for fine tuning roberta/robertuito

In [14]:
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
)
import torch
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [15]:
# we use this function to find the majority element in a list
def find_majority(k):
    myMap = {}
    maximum = ("", 0)  # (occurring element, occurrences)
    for n in k:
        if n in myMap:
            myMap[n] += 1
        else:
            myMap[n] = 1

        # Keep track of maximum on the go
        if myMap[n] > maximum[1]:
            maximum = (n, myMap[n])

    return maximum

In [16]:
l = [1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3]
find_majority(l)

(1, 5)

In [17]:
# setting up the model and the tokenizer
xlm_roberta = "xlm-roberta-base"
xlm_tokenizer = AutoTokenizer.from_pretrained(xlm_roberta)
xlm_model = AutoModelForSequenceClassification.from_pretrained(
    xlm_roberta, num_labels=2
)

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense

In [18]:
# defining a generaic dataset class

from torch.utils.data import Dataset


class CustomDataset(Dataset):
    def __init__(self, texts, labels, tokenizer):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(
            text,
            return_tensors="pt",
            padding="max_length",
            truncation=True,
            max_length=512,
        )
        return {
            key: torch.tensor(val[0]) for key, val in encoding.items()
        }, torch.tensor(label)

In [19]:
# we need to define a function that will take the labels and give the max occuring label
# TODO: tie breaking
def majority_vote(labels):
    majority = []
    for label in labels:
        majority.append(find_majority(label)[0])
    return majority

In [20]:
def label_convertor(labels):
    return [0 if label == "NO" else 1 for label in labels]


def label_deconvertor(labels):
    return ["NO" if label == 0 else "YES" for label in labels]

In [21]:
train_labels1 = train["labels_task1"]
train_lang = train["lang"]
train_text = train["tweet"]

In [22]:
train_labels_pre = majority_vote(train_labels1)

In [23]:
train_labels = label_convertor(train_labels_pre)

In [24]:
def split_lang(texts, langs, labels):
    espanol, esp_labels = [], []
    english, eng_labels = [], []
    for text, lang, label in zip(texts, langs, labels):
        if lang == "es":
            espanol.append(text)
            esp_labels.append(label)
        else:
            english.append(text)
            eng_labels.append(label)
    return english, eng_labels, espanol, esp_labels

In [25]:
english_train, eng_train_labels, espanol_train, esp_train_labels = split_lang(
    train_text, train_lang, train_labels
)

In [26]:
# loading the validation datset
validation = pd.read_json("dataset/dev/EXIST2023_dev.json", orient="index")

In [27]:
validation = validation.sample(frac=1).reset_index(drop=True)

In [28]:
valid_labels1 = validation["labels_task1"]
valid_lang = validation["lang"]
valid_text = validation["tweet"]

In [29]:
valid_labels_pre = majority_vote(valid_labels1)

In [30]:
valid_labels = label_convertor(valid_labels_pre)

In [31]:
english_valid, eng_valid_labels, espanol_valid, esp_valid_labels = split_lang(
    valid_text, valid_lang, valid_labels
)

## We train xlm-roberta first

In [32]:
# we create a CustomDataset object from the training and validation data. ignore lang for now
xlm_train = CustomDataset(train_text, train_labels, xlm_tokenizer)
xlm_valid = CustomDataset(valid_text, valid_labels, xlm_tokenizer)

In [33]:
# from transformers import AutoTokenizer, AutoModelForMaskedLM

# tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")
# model = AutoModelForMaskedLM.from_pretrained("xlm-roberta-base")

# # prepare input
# text = "Replace me by any text you'd like."
# encoded_input = tokenizer(text, return_tensors="pt")

# # forward pass
# output = model(**encoded_input)

In [34]:
# encoded_input

In [35]:
# hyperpaprameters. tune these somehow
xlm_training_args = TrainingArguments(
    output_dir="./output",
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    evaluation_strategy="steps",
    eval_steps=10,
    save_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
)


def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, preds, average="binary"
    )
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}


xlm_trainer = Trainer(
    model=model,
    args=xlm_training_args,
    train_dataset=xlm_train,
    eval_dataset=xlm_valid,
    compute_metrics=compute_metrics,
)

In [36]:
# xlm_trainer.train()

In [37]:
from transformers import AutoTokenizer, AutoModelForMaskedLM, AutoConfig
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import pandas as pd

In [38]:
def calculate_accuracy(preds, labels):
    preds = torch.argmax(preds, dim=1)
    correct = (preds == labels).sum().item()
    return correct / len(labels)

In [39]:
epochs = 3
batch_size = 8
learning_rate = 2e-5
weight_decay = 0.01

In [40]:
model_name = (
    "xlm-roberta-base"  # or any other pre-trained MLM model available on Hugging Face
)
tokenizer = AutoTokenizer.from_pretrained(model_name)
mlm_model = AutoModelForMaskedLM.from_pretrained(model_name)
config = AutoConfig.from_pretrained(model_name)

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file config.json from cache at /Users/ravimaith/.cache/huggingface/hub/models--xlm-roberta-base/snapshots/77de1f7a7e5e737aead1cd880979d4f1b3af6668/config.json
Model config XLMRobertaConfig {
  "_name_or_path": "xlm-roberta-base",
  "architectures": [
    "XLMRobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.22.2",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 250002
}

loading file sen

In [49]:
class CustomClassifier(nn.Module):
    def __init__(self, base_model, num_labels):
        super(CustomClassifier, self).__init__()
        self.base_model = base_model
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, num_labels)

    def forward(self, input_ids, attention_mask=None):
        base_model_output = self.base_model(input_ids)
        pooled_output = base_model_output[0][
            :, 0, :
        ]  # Take the pooled output (CLS token representation)
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        return logits

In [42]:
num_labels = 2  # Adjust to the number of classes in your problem
model = CustomClassifier(mlm_model, num_labels)

In [43]:
class CustomDataset(Dataset):
    def __init__(self, texts, labels, tokenizer):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(
            text,
            return_tensors="pt",
            padding="max_length",
            truncation=True,
            max_length=512,
        )
        return {
            key: torch.tensor(val[0]) for key, val in encoding.items()
        }, torch.tensor(label)

In [44]:
train_dataset = CustomDataset(train_text, train_labels, tokenizer)
val_dataset = CustomDataset(valid_text, valid_labels, tokenizer)

In [51]:
from torch.optim import AdamW

device = torch.device("mps" if torch.cuda.is_available() else "cpu")
model.to(device)
optimizer = AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
total_steps = len(train_text) * epochs // batch_size
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=total_steps)

In [52]:
from torch.utils.data import DataLoader

model.train()
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    print("-" * 30)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    # Training loop
    train_loss = 0.0
    train_accuracy = 0.0
    for i, (inputs, labels) in enumerate(train_loader):
        inputs, labels = {k: v.to(device) for k, v in inputs.items()}, labels.to(device)
        optimizer.zero_grad()
        logits = model(input_ids=inputs["input_ids"])
        loss = nn.CrossEntropyLoss()(logits, labels)
        loss.backward()
        optimizer.step()
        scheduler.step()
        train_loss += loss.item()
        train_accuracy += calculate_accuracy(logits, labels)

    avg_train_loss = train_loss / len(train_loader)
    avg_train_accuracy = train_accuracy / len(train_loader)
    print(f"Train loss: {avg_train_loss}, Train accuracy: {avg_train_accuracy}")

    # Validation loop
    model.eval()
    val_loss = 0.0
    val_accuracy = 0.0
    with torch.no_grad():
        for i, (inputs, labels) in enumerate(val_loader):
            inputs, labels = {k: v.to(device) for k, v in inputs.items()}, labels.to(
                device
            )
            logits = model(input_ids=inputs["input_ids"])
            loss = nn.CrossEntropyLoss()(logits, labels)
            val_loss += loss.item()
            val_accuracy += calculate_accuracy(logits, labels)

    avg_val_loss = val_loss / len(val_loader)
    avg_val_accuracy = val_accuracy / len(val_loader)
    print(f"Validation loss: {avg_val_loss}, Validation accuracy: {avg_val_accuracy}")

Epoch 1/3
------------------------------


/var/folders/fv/2rfdp87n0299b4fs27y2y18c0000gn/T/ipykernel_19735/1631109833.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  key: torch.tensor(val[0]) for key, val in encoding.items()


RuntimeError: mat1 and mat2 shapes cannot be multiplied (8x250002 and 768x2)